# Obliczanie zwrotu z inwestycji


W tej części projektu, na podstawie sygnałów generwanych przez poprzedni kod pythona obliczone zostaną dzienne stopy zwrotu z rozważanych strategii

In [2]:
### BIBLIOTEKI ###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### ŚCIEŻKI ##
file_path1 = "C:/Users/olgas/OneDrive/Documents/GitHub/Hight-turnover-effect-GPW/rates_data/rates_data.xlsx"
file_path2 = "C:/Users/olgas/OneDrive/Documents/GitHub/Hight-turnover-effect-GPW/prices_cleaned.xlsx"
file_path3 = "C:/Users/olgas/OneDrive/Documents/GitHub/Hight-turnover-effect-GPW/GPW capitalization USD mies.xlsx"
file_path4 = "C:/Users/olgas/OneDrive/Documents/GitHub/Hight-turnover-effect-GPW/zestawienie nazw.xlsx"



## Wczytywanie danych


Sygnały są zapisane w 15 strategiach (każda w osobnym arkuszu w pliku excel). Odczytuje wszytskiie do słownika ramek danych


In [3]:
prices = pd.read_excel(file_path2)

prices.iloc[:, 1:] = prices.iloc[:, 1:].applymap(lambda x: pd.to_numeric(x, errors='coerce'))

capitalization = pd.read_excel(file_path3)

accuracy = pd.read_excel(file_path4)


sheets_dict = pd.read_excel(file_path1, sheet_name=None)

sygnals = {} # Pusty słownik do zappisywania 
for sheet_name, dataframe in sheets_dict.items():
    sygnals[sheet_name] = dataframe

# Odwoływanie sie do konkretnych ramek danych -> sygnals['rates_10_1']

C:\Users\olgas\AppData\Local\Temp\ipykernel_15756\322302912.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  prices.iloc[:, 1:] = prices.iloc[:, 1:].applymap(lambda x: pd.to_numeric(x, errors='coerce'))


## Dzienne stopy zwrotu 

In [4]:
def daily_returns(df):
    result = pd.DataFrame(columns=df.columns)  # Nowa ramka danych do przechowywania wyników

    for col in df.columns:
        if col == 'Date':
            
            result['Date'] = df['Date'][1:].reset_index(drop=True)
        else:
            # Oblicz dzienne stopy zwrotu
            rates_values = []  # Pusta lista na dzienne stopy zwrotu
            for i in range(len(df) - 1):
                rate = (df[col].iloc[i + 1] - df[col].iloc[i]) / df[col].iloc[i]
                rates_values.append(rate)  # Dodaj wynik do listy

            result[col] = pd.Series(rates_values).reset_index(drop=True)

    return result

daily_returns_df = daily_returns(prices)

## Ujednolicenie kolumn 


W badaniu potrzeba ujednolicić kolumny w ramkach danych z kapitalizacją i cenami i sygnałami


In [5]:
columns_to_remove = accuracy.loc[accuracy['czy zgodnosc'].isnull(), 'Profesora'].tolist() # Kolumny których nie ma w mojej tabeli do usuniecia 

capitalizationv2 = capitalization.drop(columns=columns_to_remove) # usunięcie wartości których ja u siebie nie mam 

capitalizationv3 = capitalizationv2[prices.columns] # ustawienie w tej samej kolejności kolumny



## Okres inwestycji - 1 dzień 


W przypadku okresu sprzedaży równego 1 dzień, nie trzeba liczyć średnich dziennych stóp zwrotu z uwzględnieniem tego okresu 


In [62]:
def one_day(sygnal_df, prices_df, price, kap_min, kap_df):

    
    # Upewnij się, że wszystkie daty są w formacie datetime
    sygnal_df['Year'] = pd.to_datetime(sygnal_df['Date']).dt.year
    sygnal_df['Month'] = pd.to_datetime(sygnal_df['Date']).dt.month
    kap_df['Year'] = pd.to_datetime(kap_df['Date']).dt.year
    kap_df['Month'] = pd.to_datetime(kap_df['Date']).dt.month

    # Dołącz kapitalizację na podstawie roku i miesiąca
    kap_df = kap_df.drop(columns=['Date'])  # Usuń oryginalną kolumnę Date, aby uniknąć konfliktów
    merged_df = pd.merge(
        sygnal_df,
        kap_df,
        on=['Year', 'Month'],
        how='left'
    )

    sygnal_df = sygnal_df.drop(columns=['Year', 'Month'])

    # Filtrowanie ramek
    filtered_kap = [col for col in merged_df.columns if col.endswith('_y')]
    filtered_kap = merged_df[filtered_kap]
    filtered_prices = prices_df.loc[sygnal_df.index]

    # Pusta ramka na wyniki
    result = pd.DataFrame(columns=['Date', 'Mean', 'Count'])


    # Iteracja po wierszach
    for i in range(len(sygnal_df)):
        row_sum = 0
        count = 0

        for col in sygnal_df.columns:
            if col == 'Date':
                result.loc[i, 'Date'] = sygnal_df.loc[i, 'Date']
            else:
                if not pd.isna(sygnal_df[col].iloc[i]):

                    # Sprawdzamy, czy kolumna istnieje w filtered_prices
                    if col in filtered_prices.columns:
                        # Pobieramy odpowiednią wartość kapitalizacji dla konkretnego wiersza
                        kap_value = filtered_kap.loc[i, col + '_y']

                        # Sprawdzamy warunki ceny i kapitalizacji
                        if (filtered_prices[col].iloc[i] > price and
                                kap_min <= kap_value and pd.notna(kap_value)):
                            row_sum += sygnal_df[col].iloc[i]
                            count += 1
                    else:
                        print(f"Kolumna {col} nie istnieje w filtered_prices, pomijam wiersz {i}")

        # Obliczanie średniej i zapis wyników
        mean = row_sum / count if count > 0 else 0
        result.loc[i, 'Mean'] = mean
        result.loc[i, 'Count'] = count

    return result


# Teścik


In [ ]:
# wszystkie 
results_10_1 = one_day(sygnals['rates_10_1'], prices, 0, 0, capitalizationv2)

,Date,Mean,Count
0,2002-01-17 00:00:00,0.006125,5
1,2002-01-18 00:00:00,0.001785,13
2,2002-01-21 00:00:00,0.018884,17
3,2002-01-22 00:00:00,-0.010397,7
4,2002-01-23 00:00:00,0.026858,16
...,...,...,...
5315,2023-04-24 00:00:00,-0.001813,35
5316,2023-04-25 00:00:00,0.006861,44
5317,2023-04-26 00:00:00,0.000121,50
5318,2023-04-27 00:00:00,0.017477,45


0       1.001949
1       0.962456
2       1.009389
3       1.032274
4       0.986055
          ...   
5315    1.591053
5316    1.568564
5317    1.596104
5318         NaN
5319         NaN
Name: EFK:PL, Length: 5320, dtype: float64

C:\Users\olgas\AppData\Local\Temp\ipykernel_15756\1103781018.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  capitalizationv3['Year'] = pd.to_datetime(capitalizationv3['Date']).dt.year
C:\Users\olgas\AppData\Local\Temp\ipykernel_15756\1103781018.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  capitalizationv3['Month'] = pd.to_datetime(capitalizationv3['Date']).dt.month


,01C:PL_y,01N:PL_y,06N:PL_y,08N:PL_y,11B:PL_y,13N:PL_y,14N:PL_y,1AT:PL_y,2CP:PL_y,3RG:PL_y,...,ZMT:PL_y,ZNT:PL_y,ZOC:PL_y,ZRE:PL_y,ZRW:PL_y,ZRX:PL_y,ZUE:PL_y,ZUK:PL_y,ZWC:PL_y,ZWG:PL_y
0,NaN,21872386.51,24615458.09,72907955.04,NaN,26708854.82,28296948.89,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4357592.526,NaN,NaN,NaN,739144453.5,NaN
1,NaN,21872386.51,24615458.09,72907955.04,NaN,26708854.82,28296948.89,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4357592.526,NaN,NaN,NaN,739144453.5,NaN
2,NaN,21872386.51,24615458.09,72907955.04,NaN,26708854.82,28296948.89,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4357592.526,NaN,NaN,NaN,739144453.5,NaN
3,NaN,21872386.51,24615458.09,72907955.04,NaN,26708854.82,28296948.89,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4357592.526,NaN,NaN,NaN,739144453.5,NaN
4,NaN,21872386.51,24615458.09,72907955.04,NaN,26708854.82,28296948.89,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4357592.526,NaN,NaN,NaN,739144453.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5315,15505191.29,NaN,12203776.61,10350086.58,362409066.2,NaN,NaN,433943368.9,NaN,8695466.769,...,32725025.75,NaN,NaN,15816827.53,NaN,8873762.963,28461482.6,6682456.135,NaN,NaN
5316,15505191.29,NaN,12203776.61,10350086.58,362409066.2,NaN,NaN,433943368.9,NaN,8695466.769,...,32725025.75,NaN,NaN,15816827.53,NaN,8873762.963,28461482.6,6682456.135,NaN,NaN
5317,15505191.29,NaN,12203776.61,10350086.58,362409066.2,NaN,NaN,433943368.9,NaN,8695466.769,...,32725025.75,NaN,NaN,15816827.53,NaN,8873762.963,28461482.6,6682456.135,NaN,NaN
5318,15505191.29,NaN,12203776.61,10350086.58,362409066.2,NaN,NaN,433943368.9,NaN,8695466.769,...,32725025.75,NaN,NaN,15816827.53,NaN,8873762.963,28461482.6,6682456.135,NaN,NaN
